## Manipolazione di un file GTF (Gene Transfer Format) con `Pandas`

### Importare `Pandas` e leggere il file `GTF`

In [ ]:
import pandas as pd

In [ ]:
df = pd.read_csv('./input.gtf', sep = '\t', header = None)

### Cambiare i nomi delle colonne

I nomi delle colonne devono essere:
- reference
- source
- feature
- start
- end
- score
- strand
- frame
- attributes

In [ ]:
replace_dict = {0 : 'reference', 1 : 'source', 2 : 'feature', 3 : 'start',
                4 : 'end', 5 : 'score', 6 : 'strand', 7 : 'frame', 8 : 'attributes'}

df.rename(columns = replace_dict, inplace = True)

In [ ]:
df

### Eliminare le colonne `source` e `score`

In [ ]:
df.drop(['source', 'score'], axis = 1, inplace = True)

In [ ]:
df

### Sostituire la colonna degli attributi con le due colonne  `gene` e `transcript`

La colonne `gene` e `transcript` dovranno contenere solo il nome del gene e l'ID del trascritto.

In [ ]:
import re

In [ ]:
df['transcript_id'] = df['attributes'].apply(lambda x : re.search(r'transcript_id\s+"(.+?)";', x).group(1))

In [ ]:
df['gene'] = df['attributes'].apply(lambda x : re.search(r'gene_id\s+"(.+?)";', x).group(1))

In [ ]:
df

In [ ]:
df.drop('attributes', axis = 1, inplace = True)

In [ ]:
df

### Aggiungere la colonna `length` contenente la lunghezza della *feature*

In [ ]:
df['length'] = df['end'] - df['start'] + 1

In [ ]:
df

In [ ]:
df = df.reindex(columns = ['reference', 'feature', 'start', 'end',
                      'length', 'strand', 'frame', 'gene', 'transcript_id'])

In [ ]:
df

### Rimuovere tutte le *features* di lunghezza ≤ 6

In [ ]:
df = df[df['length'] > 6]

In [ ]:
df

### Estrarre il *data frame* dei 20 esoni più lunghi e degli ultimi 20 esoni localizzati sulla *reference sequence*

In [ ]:
exon_df = df[df['feature'] == 'exon']

In [ ]:
first_df = exon_df.sort_values('length', ascending = False).head(20)

In [ ]:
second_df = exon_df.sort_values('end').tail(20)

In [ ]:
pd.merge(first_df, second_df, how = 'outer')

### Estrarre i nomi dei geni e gli identificatori dei trascritti annotati nel file GTF

In [ ]:
for gene_name in set(df['gene']):
    print(gene_name)

In [ ]:
for transcript_id in set(df['transcript_id']):
    print(transcript_id)

### Determinare, per ogni gene, la lunghezza media, massima e minima degli esoni

In [ ]:
gb = df[df['feature'] == 'exon'].groupby('gene')['length']

In [ ]:
gb.mean()

In [ ]:
gb.max()

In [ ]:
gb.min()

### Determinare la lunghezza minima degli esoni ed estrarre tutti i trascritti che contengono un esone di lunghezza minima

a) Determinare la lunghezza minima degli esoni

In [ ]:
min_length = gb.min().min()

b) Estrarre gli identificatori dei trascritti che contengono un esone di lunghezza minima

In [ ]:
mask = (df['length'] == min_length) & (df['feature'] == 'exon')
df[mask]['transcript_id'].unique()

### Contare quanti trascritti sono annotati per il gene `ARHGAP4`

In [ ]:
len(df[(df['gene'] == 'ARHGAP4') & (df['feature'] == 'exon')]['transcript_id'].unique())

### Estrarre i geni con strand `+`

In [ ]:
df[df['strand'] == '+']['gene'].unique()

### Estrarre gli esoni (distinti) del gene `ATP6AP1` in una lista di tuple (start, end)

In [ ]:
mask = (df['gene'] == 'ATP6AP1') & (df['feature'] == 'exon')
df_temp = df[mask]

In [ ]:
set([(record.start, record.end) for index, record in df[mask].iterrows()])

In [ ]:
set(zip(df_temp.start, df_temp.end))

### Contare il numero di trascritti del gene `ARHGAP4` che hanno una CDS annotata

In [ ]:
mask = (df.gene == 'ARHGAP4') & (df.feature == 'CDS')
set(df[mask]['transcript_id'])

### Estrarre lo strand del gene  `ATP6AP1`

In [ ]:
df[df.gene == 'ATP6AP1']['strand'].unique()[0]

### Determinare il trascritto che ha più esoni

In [ ]:
mask = df.feature == 'exon'
group_by_transcript_items = df[mask].groupby('transcript_id').groups.items()

group_by_transcript_items

In [ ]:
sorted(map(lambda x : (len(x[1]), x[0]) , group_by_transcript_items), reverse = True).pop(0)[1]

### Estrarre per ogni trascritto del gene  `ARHGAP4` la lista delle tuple (start, end) dei suoi esoni ordinate per start crescente

In [ ]:
mask = (df.gene == 'ARHGAP4') & (df.feature == 'exon')
for transcript_id in df[mask].groupby('transcript_id').groups:
    print(transcript_id)
    index_list = df[mask].groupby('transcript_id').groups[transcript_id]
    print(sorted(zip(df.loc[index_list]['start'], df.loc[index_list]['end'])))

### Determinare, per ogni gene e ogni trascritto il numero di esoni che lo compongono

### Determinare per ogni gene, la lunghezza del suo *locus*